In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
import pickle

with open('alldata.pkl', 'rb') as in_file:
    data = pickle.load(in_file)


In [33]:
biosensor = []
masks = []

for d in data:
    biosensor.append(d['biosensor'])
    masks.append(d['mask'])
